# Aluminum Rod Simulation

A program to simulate an isolated rod being heated, and undergoing heat loss via convection and radiation. Considers conduction through the rod.

## Units

- Length: m
- Temperature: K
- Mass: kg
- Energy: J
- Time: s

## Imports

In [1]:
import math

import numpy as np

import matplotlib.pyplot as plt


## Physical Constants

In [2]:
sigma = 5.67 * 10 ** (-8) # Stefan-Boltzmann, W / (m^2 K^4)

## Auxiliary Functions

In [3]:
def celsius_to_kelvin(T_C):
    return T_C + 273.15

## Simulation Parameters + Geometric Properties (customize)

In [4]:
# rod length
L = 0.3 # m

# dt + N slices -> dx
dt = 0.01 # s

N = 60
dx = L / N # m

r = 0.01 # m

cross_A = math.pi * r ** 2 # m^2

surface_A = np.array([2 * math.pi * r * dx] * N)
surface_A[0] += cross_A
surface_A[N-1] += cross_A

## Ambient Properties (customize)

In [5]:
T_amb = celsius_to_kelvin(20) # K
T_0 = celsius_to_kelvin(20)

## Thermal Behaviour

### Initial Temperature Distribution (customize)

In [6]:
def init_temperature():
    return np.array([T_0] * N)

### Power Input (customize)

In [7]:
def power_input(T, P, Pin):
    P[0] += Pin

### Conduction

#### Thermo Function

In [8]:
def conduction(T, P, k):
    
    # conduction temperatures = [T_left T T_right], copy
    # the endpoints have the effect of only conducting within the bar, not to/from the outside
    T_conduct = np.array([T[0]] + np.ndarray.tolist(T) + [T[N-1]])
    
    # P_net = P_in - P_out = kA/L ((T[n-1] - T[n]) - (T[n] - T[n+1])) = kA/L (T[n-1] - 2T[n] + T[n+1])
    P += (k * cross_A / dx) * (T_conduct[0:N] - 2*T_conduct[1:N+1] + T_conduct[2:N+2])
    

### Convection

#### Thermo Function

In [9]:
def convection(T, P, k_c):
    P -= surface_A * k_c * (T - T_amb)

### Radiation

#### Thermo Function

In [10]:
def radiation(T, P, e):
    P -= surface_A * e * sigma * (T ** 4 - T_amb ** 4)

### Calorimetry

#### Parameters (customize)

In [11]:
rho = 2700 # kg/m^3

#### Thermo Function

In [12]:
def calorimetry(T, P, c):
    
    Q = P * dt # P and dt small
    
    dm = rho * cross_A * dx # dx small
    dT = Q / (c * dm)
    
    T += dT

### Master

In [13]:
def run_thermal_iter(T, P, Pin, k, k_c, e, c):
    
    # compute Power
    power_input(T, P, Pin)
    conduction(T, P, k)
    convection(T, P, k_c)
    radiation(T, P, e)
    
    # update Temperature from Power
    calorimetry(T, P, c)
    

## Plotting

### Captions

In [14]:
#caption_main = ""

### Additional Parameters

In [15]:
x_points = np.array(range(N)) * dx

### Plot Functions

In [16]:
def plot_main(x_vals, y_vals):

    fig, ax = plt.subplots()
    
    ax.set_title("Steady State Temperature Distribution in Aluminum Rod")

    ax.set_xlabel("x-coordinate (m)")
    ax.set_ylabel("Tepmerature (K)")

    ax.plot(x_vals, y_vals, 'bx', label='Simulated Data')

    ax.legend()
    
    #fig.text(0.5, -0.08, caption, ha='center')
    
    fig.savefig("foo.png")

## Simulation

### Simulation Auxiliary Functions

In [17]:
def init_power():
    return np.zeros(N)

### Execute Simulation

In [18]:
def run_sim(t_max, t_saves, thermo_params):
    
    """
    """
    
    ## Flow: in every iteration, find net power flow distribution from temperature distribution in previous iteration
    ##       use that to get temperature temperature distribution for the next iteration
    
    t = 0
    
    T = init_temperature()
    
    saved_data = []
    
    while t < t_max:

        # iteration init
        P = init_power()
        
        # run
        run_thermal_iter(T, P, *thermo_params)
        
        t += dt
    
    # Plot temperature distribution
    plot_main(x_points, T)
    

# Curve-fit Optimization

## Single Curve-Fit

In [19]:
def single_curve_fit(Pin, k, k_c, e, c):
    pass